In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from ilxutils.ttl2pd import *
import difflib

def degrade(var):
    def helper(s):
        return str(re.sub("\(|\)|'|,", "", s).lower().strip())
    if type(var)!=list:
        if var:
            return helper(var)
        else:
            return var
    else:
        return [helper(v) if v else v for v in var]
    
def cj(data, output):
    with open(output+'.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)
    print('Complete')
def oj(filepath):
    with open(filepath+'.json', 'r') as infile:
        return json.load(infile)
        
def cp(data, output):
    with open(output+'.pickle', 'wb') as outfile:
        pickle.dump(data, outfile)
    print('Complete')
def op(filepath):
    with open(filepath+'.pickle', 'rb') as infile:
        return pickle.load(infile)

In [ ]:
'''Need to open nda types and see if they are subjs in the rdflib nif ontologies'''

In [3]:
args = read_args(api_key= p.home() / 'keys/production_api_scicrunch_key.txt', db_url= p.home() / 'keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [5]:
cp(annos, '../../../ilx_db_anno_backup') #need to make a script to back up all tables
cp(ex, '../../../ilx_db_ex_backup')
cp(terms, '../../../ilx_db_terms_backup')

Complete
Complete
Complete


In [33]:
annos = op('../../../ilx_db_anno_backup')
ex = op('../../../ilx_db_ex_backup')
terms = op('../../../ilx_db_terms_backup')

In [5]:
terms = sql.get_terms()
annos = sql.get_annotations()
ex = sql.get_existing_ids()

In [72]:
annos[annos.id == 6908]

,id,tid,annotation_tid,value,term_label,term_ilx,annotation_tid,annotation_label,annotation_ilx
7035,6908,15077,15074,filterElement : None,subjectkey (12-Item Grit Scale),ilx_0115074,15074,hasConstraint,ilx_0115071


In [34]:
terms[terms.label == 'grade_highed (AASPIRE Health Integration)']

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
15513,15518,ilx_0115515,0,0,0,0,grade_highed (AASPIRE Health Integration),Grade/Highest education,,cde,4,0,1,1515202052,1524592193


In [6]:
annos.head()

,id,tid,annotation_tid,value,term_label,term_ilx,annotation_label,annotation_ilx
0,38201,182,15074,categories : Questionnaire,Abnormal Involuntary Movement Scale,ilx_0100181,hasConstraint,ilx_0115071
1,38202,182,15074,status : Published,Abnormal Involuntary Movement Scale,ilx_0100181,hasConstraint,ilx_0115071
2,38208,182,15074,dataType : Clinical Assessments,Abnormal Involuntary Movement Scale,ilx_0100181,hasConstraint,ilx_0115071
3,38216,182,15074,sources : NDAR; NDCT; RDoC,Abnormal Involuntary Movement Scale,ilx_0100181,hasConstraint,ilx_0115071
4,38220,182,15074,modifiedDate : 1515012631513,Abnormal Involuntary Movement Scale,ilx_0100181,hasConstraint,ilx_0115071


In [4]:
ex.head()

,id,tid,curie,iri,preferred,ilx,type,label,definition
0,341811,1,NLXWIKI:nifext_5214,http://neurolex.org/wiki/nifext_5214,1,ilx_0100000,term,(R)N6-phenylisopropyladenosine,
1,341812,1,NIFEXT:5214,http://uri.neuinfo.org/nif/nifstd/nifext_5214,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,
2,341813,1,ILX:0100000,http://uri.interlex.org/base/ilx_0100000,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,
3,341814,2,NLXWIKI:T3D0044,http://neurolex.org/wiki/T3D0044,1,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...
4,341815,2,T3D:0044,http://t3db.org/toxins/T3D0044,0,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...


In [56]:
set(annos[annos.annotation_label.str.contains('type')].annotation_label)

{'Variable type'}

In [13]:
'''check if all 2nd tier has NDA.CDE exs'''
excheck = defaultdict(list)
for i, row in ex.iterrows():
    excheck[row.tid].append(row.curie.split(':', 1)[0])

In [14]:
for tid in oj('../../../nda_tier2_term_ids'):
    if 'NDA.CDE' not in excheck[tid]:
        print(tid, excheck[tid])
        break

In [19]:
for i, row in annos.iterrows():
    print({**row, **{'test':1}})
    break

{'id': 38201, 'item_id': 182, 'label': 'Abnormal Involuntary Movement Scale', 'ilx': 'ilx_0100181', 'anno_tid': 15074, 'anno_label': 'hasConstraint', 'anno_ilx': 'ilx_0115071', 'tid': 182, 'annotation_tid': 15074, 'value': 'categories : Questionnaire', 'test': 1}


In [8]:
stak = { #second tier annotation keys
    'condition':True,
    'filterElement':True,
    'id':True,
    'notes':True,
    'required':True,
    'size':True,
    'type':True,
    'valueRange':True,
}
def contains_key(string):
    if ':' in string:
        sk, sval = string.split(':', 1)[0].strip(), string.split(':', 1)[1].strip() #string key
        if stak.get(sk):
            return sk, sval
    return False, False

In [8]:
nda_tier2_term_ids = []
for i, row in annos[annos.annotation_label == 'hasConstraint'].iterrows(): ## get second tier annotations only -> anno_label == 'hasConstraint'
    key_type, key_val = contains_key(row.value)
    if key_val:
        nda_tier2_term_ids.append(row.tid)
cj(nda_tier2_term_ids, '../../../nda_tier2_term_ids')

Complete


In [76]:
udata = []
adata = []
cdata = []
ddata = []
tudata = []
visited = {}
filterElementID = 304437
for i, row in annos[annos.annotation_label == 'hasConstraint'].iterrows(): ## get second tier annotations only -> anno_label == 'hasConstraint'
    if i % 200000 == 0: print(i)
    key_type, key_val = contains_key(row.value) ## filter each term by their annotation values by key : value
    
    if key_type == 'condition':
        cdata.append({'condition':key_val})
    
    elif key_type == 'filterElement':
        #cdata.append({'filterElement':key_val})
        ddata.append(row.id)
        if key_val == "None":
            adata.append({'tid':row.tid, 'annotation_tid':filterElementID, 'value':False})
        elif key_val == '1':
            adata.append({'tid':row.tid, 'annotation_tid':filterElementID, 'value':True})
        else:
            sys.exit(key_val)
            
    #elif key_type == 'id':
    #    ddata.append(row.id)
        
    #elif key_type == 'notes':
    #    if key_val != "None" and not visited.get(row.tid):
    #        tudata.append({'id':row.tid, 'ilx':row.term_ilx, 'comment':key_val})
    #        visited[row.tid] = True
    #    ddata.append(row.id)
    
    elif key_type == 'required':
        cdata.append({'required':key_val})
    
    elif key_type == 'size':
        if key_val != 'None':
            print(key_val, row)
            sys.exit()
        cdata.append({'size':key_val})
    
    elif key_type == 'type':
        cdata.append({'type':key_val})

    elif key_type == 'valueRange':
        if '::' in key_val:
            minValue, maxValue = key_val.split('::')[0].split(), key_val.split('::')[1].split()
            #ddata.append(row.id)
            #adata.append({'tid':row.tid, 'annotation_tid':row.annotation_tid, 'value':minValue})
            #adata.append({'tid':row.tid, 'annotation_tid':row.annotation_tid, 'value':maxValue})
        else:
            
            cdata.append({'valueRange':key_val})

## condition -> ?
## filterElement -> ?
## delete Id if it is preferred already
## notes -> move to notes
## required -> get list of options this has
## size -> get ex that isn't none
## type -> porb
## valueRange -> seperate into minValue and maxValue

0
20 id                                                 6917
tid                                               15078
annotation_tid                                    15074
value                                         size : 20
term_label          src_subject_id (12-Item Grit Scale)
term_ilx                                    ilx_0115075
annotation_tid                                    15074
annotation_label                          hasConstraint
annotation_ilx                              ilx_0115071
Name: 7045, dtype: object


SystemExit: 

/home/troy/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [78]:
len("src_subject_id")

14

In [29]:
ddata[0]

6912

In [71]:
cj(sorted(ddata), '../../../nda_annotation_filterElement_delete')
cj(adata, '../../../nda_annotation_filterElement_add')
#cj(tudata, '../../../nda_term_update')
#cj(udata, '../../../nda_annotation_update')
#cj(cdata, '../../../nda_annotation_check')

Complete
Complete


In [67]:
data = [{'term':'filterElement', 'type':'annotation'}]
output = sci.addTerms(data, LIMIT=1, crawl=True, _print=False)#304437

In [68]:
output

[{'id': 304437,
  'orig_uid': None,
  'uid': None,
  'orig_cid': None,
  'cid': None,
  'ilx': 'ilx_0381352',
  'label': 'filterElement',
  'type': 'annotation',
  'definition': None,
  'comment': None,
  'version': 1,
  'status': 0,
  'display_superclass': 1,
  'orig_time': 1527632472,
  'time': 1527632472,
  'synonyms': [],
  'superclasses': [],
  'existing_ids': [{'id': '2019978',
    'tid': '304437',
    'curie': 'ILX:0381352',
    'iri': 'http://uri.interlex.org/base/ilx_0381352',
    'curie_catalog_id': '3',
    'version': '1',
    'time': '1527632472',
    'preferred': '1'}],
  'relationships': [],
  'mappings': [],
  'annotations': [],
  'annotation_type': 'text',
  'ontologies': []}]

In [6]:
cdata = oj('../../../nda_annotation_check')

In [74]:
'''explainer'''
ctypes = defaultdict(set)
for d in cdata:
    ctypes[list(d)[0]].add(list(d.values())[0])

In [75]:

#ctypes['codition'] #random code; ex ->  'famhx_1   == 1 &&  fam_history_12_yes_no   == 1 &&  fhx_3h_younger_p  > 4 &&  fhx_3h_younger_p  != 999',
ctypes['size'] #ints with no measurement specifics 
#ctypes['filterElement'] #either 1 or None, makes me think its really 0 or 1 for true or false
#ctypes['required'] # {'Conditional', 'Optional', 'Recommended', 'Required'}
#ctypes['type'] #{'Date', 'File', 'Float', 'GUID', 'Integer', 'String', 'Thumbnail'}
#ctypes['valueRange']

{'1',
 '10',
 '100',
 '1000',
 '1020',
 '1024',
 '1050',
 '12',
 '120',
 '1200',
 '1250',
 '13',
 '15',
 '150',
 '1500',
 '16',
 '160',
 '165',
 '175',
 '2',
 '20',
 '200',
 '2000',
 '21',
 '225',
 '240',
 '25',
 '250',
 '255',
 '2750',
 '29',
 '290',
 '3',
 '30',
 '300',
 '32',
 '33',
 '35',
 '350',
 '3500',
 '360',
 '4',
 '40',
 '400',
 '4000',
 '425',
 '45',
 '450',
 '5',
 '50',
 '500',
 '51',
 '510',
 '512',
 '55',
 '6',
 '60',
 '600',
 '62',
 '65',
 '75',
 '8',
 '80',
 '800',
 'None'}

In [54]:
import subprocess as sb
for typ in ['Date', 'File', 'Float', 'GUID', 'Integer', 'String', 'Thumbnail']:
    datatype = "xsd:"+typ
    sbinput="grep -nir '{}' /home/troy/Desktop/work/NIF-Ontology/".format(datatype)
    print(sbinput)
    output = sb.getoutput(sbinput)
    if output and typ not in ['String']:
        print(output)

grep -nir 'xsd:Date' /home/troy/Desktop/work/NIF-Ontology/
grep -nir 'xsd:File' /home/troy/Desktop/work/NIF-Ontology/
grep -nir 'xsd:Float' /home/troy/Desktop/work/NIF-Ontology/
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:511:    rdfs:range xsd:float ;
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:531:    rdfs:range xsd:float ;
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:549:    rdfs:range xsd:float ;
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:557:    rdfs:range xsd:float ;
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:566:    rdfs:range xsd:float ;
/home/troy/Desktop/work/NIF-Ontology/ttl/unused/SAO-CORE_properties.ttl:758:    rdfs:range xsd:float ;
grep -nir 'xsd:GUID' /home/troy/Desktop/work/NIF-Ontology/
grep -nir 'xsd:Integer' /home/troy/Desktop/work/NIF-Ontology/
grep -nir 'xsd:String' /home/troy/Desktop/work/NIF-Ontology/
grep -nir 'xsd:Thumbna

In [51]:
from rdflib.namespace import XSD
XSD.troy

rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#troy')